## __get 2cd data

In [2]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import datetime

In [3]:
b1_data = pd.read_excel(r'..\竞赛发布数据\表1-患者列表及临床信息.xlsx')
train_ID = b1_data['Unnamed: 0'].tolist()
new_b2_data = pd.read_csv(r'..\竞赛发布数据\表2-long.csv', dtype = str)
new_b2_data = new_b2_data[new_b2_data['ID'].isin(train_ID)]
new_b2_data['ED_volume'] = [float(x) for x in new_b2_data['ED_volume']]
new_b2_data = new_b2_data[['ID', 'follow_up_count', '流水号', 'ED_volume']].copy()
new_b2_data['ED_volume'] = [float(x) for x in new_b2_data['ED_volume']]

## 根据ID，连接其发病时间
b1_data = pd.read_excel(r'..\竞赛发布数据\表1-患者列表及临床信息.xlsx', dtype = str)
b1_data = b1_data[['Unnamed: 0', '发病到首次影像检查时间间隔']]
b1_data.columns = ['ID', '发病到首次影像检查时间间隔']
b1_data['发病到首次影像检查时间间隔'] = [float(x) for x in b1_data['发病到首次影像检查时间间隔']]
new_b2_data = pd.merge(new_b2_data, b1_data, left_on = "ID", right_on = "ID", how = "left")

## 根据流水号，连接其流水号发生时间
fb1_data = pd.read_csv(r'..\竞赛发布数据\附表1-long.csv', dtype = str)
tmp1, tmp2 = fb1_data[['sub_code', '入院首次检查流水号', '入院首次检查时间点']].copy(), fb1_data[['sub_code', '流水号', '时间点']].copy()
tmp1.columns = ['ID', '流水号', '时间点']
tmp2.columns = ['ID', '流水号', '时间点']
fb1_data = pd.concat([tmp1, tmp2], axis = 0)
fb1_data = fb1_data.drop_duplicates()
new_b2_data = pd.merge(new_b2_data, fb1_data, left_on = ["ID", "流水号"], right_on = ["ID", "流水号"], how = "left")
new_b2_data['时间点'] = [datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ) for x in new_b2_data['时间点']]

new_b2_data['first_examination'] = 0
for i_row in tqdm(range(len(new_b2_data))):
    tmp_id = new_b2_data.loc[i_row, 'ID']
    tmp_df = new_b2_data[new_b2_data['ID'] == tmp_id]
    tmp_df = tmp_df[tmp_df['follow_up_count'] == '0']['时间点'].values[0]
    
    new_b2_data.loc[i_row, 'first_examination'] = pd.Timestamp(tmp_df)        ## 这里的tmp_df为时间的值
    
    
new_b2_data['time_from_disease'] = new_b2_data['时间点'] - new_b2_data['first_examination']
new_b2_data['time_from_disease'] = [x.days * 24 + x.seconds / 3600 for x in new_b2_data['time_from_disease']]
new_b2_data['time_from_disease'] = new_b2_data['time_from_disease'] + new_b2_data['发病到首次影像检查时间间隔']

## 异常值处理
def three_sigma(array):
    mu = np.mean(array)
    sigma = np.std(array)
    list_res = []
    for i in array:
        if i > mu + 3 * sigma:
            list_res.append(mu + 3 * sigma)
        elif i < mu - 3 * sigma:
            list_res.append(mu - 3 * sigma)
        else:
            list_res.append(i)
    return list_res
new_b2_data['ED_volume'] = three_sigma(new_b2_data['ED_volume'])
new_b2_data

100%|██████████| 593/593 [00:00<00:00, 1084.12it/s]
C:\Users\rukel\AppData\Local\Temp\ipykernel_6532\4111732922.py:35: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  new_b2_data['time_from_disease'] = new_b2_data['时间点'] - new_b2_data['first_examination']


,ID,follow_up_count,流水号,ED_volume,发病到首次影像检查时间间隔,时间点,first_examination,time_from_disease
0,sub001,0,20161212002136,48919.000000,2.5,2016-12-12 23:32:54,2016-12-12 23:32:54,2.500000
1,sub001,1,20161213000009,57898.000000,2.5,2016-12-13 05:19:00,2016-12-12 23:32:54,8.268333
2,sub001,2,20161218000100,81747.000000,2.5,2016-12-18 09:09:24,2016-12-12 23:32:54,132.108333
3,sub001,3,20161223001020,102313.772777,2.5,2016-12-23 16:47:09,2016-12-12 23:32:54,259.737500
4,sub001,4,20161230000574,102313.772777,2.5,2016-12-30 14:35:12,2016-12-12 23:32:54,425.538333
...,...,...,...,...,...,...,...,...
588,sub158,3,20200423000871,64243.000000,2.8,2020-04-23 11:31:00,2020-04-10 17:40:00,308.650000
589,sub159,0,20200218000582,14430.000000,3.0,2020-02-18 15:29:00,2020-02-18 15:29:00,3.000000
590,sub159,1,20200219000588,19019.000000,3.0,2020-02-19 15:01:00,2020-02-18 15:29:00,26.533333
591,sub160,0,20200821002584,18622.000000,4.0,2020-08-21 22:32:00,2020-08-21 22:32:00,4.000000


In [4]:
new_b2_data_minfollow = new_b2_data.copy()
new_b2_data_minfollow = new_b2_data_minfollow.groupby('ID').min('follow_up_count')
new_b2_data_minfollow['ID'] = new_b2_data_minfollow.index
new_b2_data_minfollow.index = pd.RangeIndex(len(new_b2_data_minfollow))
new_b2_data_minfollow = new_b2_data_minfollow[['ID', 'time_from_disease']]
new_b2_data_minfollow = pd.merge(new_b2_data_minfollow, new_b2_data, left_on=['ID', 'time_from_disease'], right_on=['ID', 'time_from_disease'], how='left')
new_b2_data_minfollow = new_b2_data_minfollow[['ID', 'time_from_disease', 'follow_up_count', 'ED_volume']]
new_b2_data_minfollow.columns = ['ID', 'first_time_from_disease', 'first_follow_up_count', 'first_ED_volume']
new_b2_data_minfollow

,ID,first_time_from_disease,first_follow_up_count,first_ED_volume
0,sub001,2.5,0,48919.0
1,sub002,3.0,0,23526.0
2,sub003,2.0,0,32621.0
3,sub004,1.0,0,22191.0
4,sub005,5.0,0,47392.0
...,...,...,...,...
155,sub156,5.5,0,8968.0
156,sub157,0.5,0,409.0
157,sub158,2.8,0,17948.0
158,sub159,3.0,0,14430.0


In [5]:
new_b2_data_maxfollow = new_b2_data.copy()
new_b2_data_maxfollow = new_b2_data_maxfollow.groupby('ID').max('follow_up_count')
new_b2_data_maxfollow['ID'] = new_b2_data_maxfollow.index
new_b2_data_maxfollow.index = pd.RangeIndex(len(new_b2_data_maxfollow))
new_b2_data_maxfollow = new_b2_data_maxfollow[['ID', 'time_from_disease']]
new_b2_data_maxfollow = pd.merge(new_b2_data_maxfollow, new_b2_data, left_on=['ID', 'time_from_disease'], right_on=['ID', 'time_from_disease'], how='left')
new_b2_data_maxfollow = new_b2_data_maxfollow[['ID', 'time_from_disease', 'follow_up_count', 'ED_volume']]
new_b2_data_maxfollow.columns = ['ID', 'last_time_from_disease', 'last_follow_up_count', 'last_ED_volume']
new_b2_data_maxfollow

,ID,last_time_from_disease,last_follow_up_count,last_ED_volume
0,sub001,425.538333,4,102313.772777
1,sub002,1119.191944,4,0.000000
2,sub003,39.595833,2,49767.000000
3,sub004,227.501111,3,5204.000000
4,sub005,97.943056,2,12898.000000
...,...,...,...,...
155,sub156,293.317222,3,12709.000000
156,sub157,20.066667,3,29545.000000
157,sub158,308.650000,3,64243.000000
158,sub159,26.533333,1,19019.000000


In [6]:
new_b2_data = pd.merge(new_b2_data_minfollow, new_b2_data_maxfollow, left_on='ID', right_on='ID', how='left')
new_b2_data.to_csv('..' + r'\竞赛发布数据\2cd_Data_checkpoint251648.csv')